In [50]:
from lxml import etree
import pandas as pd
pd.set_option('display.max_columns', None)

In [51]:
# Generate track dictionaries
xml_path = '/Users/kevinmarlis/Developer/Music Analytics/Library.xml'
tracks = []
track_dicts = []

for event, element in etree.iterparse(xml_path):
    if element.text == 'Tracks':
        results = element.getnext()

for element in results:
    if element.tag == 'dict':
        tracks.append(element)

for item in tracks:
    track = {}
    for element in item:
        if element.tag == 'key':
            track[element.text] = element.getnext().text
    track_dicts.append(track)

In [52]:
df = pd.DataFrame(track_dicts)
cols_to_drop = ['Kind', 'Size', 'Disc Number', 'Disc Count',
       'Track Number', 'Date Modified', 'Bit Rate',
       'Sample Rate', 'Artwork Count', 'Track Type', 'Location', 'File Folder Count', 
       'Library Folder Count', 'Normalization', 'Volume Adjustment', 
       'Play Date', 'Apple Music', 'Movement Name', 'Grouping',
       'Movement Number', 'Movement Count', 'Part Of Gapless Album', 'Comments', 'Album Loved', 'Matched', 'BPM',
       'Playlist Only', 'Explicit', 'Sort Composer', 'Clean', 'Protected',
       'File Type', 'Track Count']
df = df.drop(cols_to_drop, axis=1)
df.head()

,Track ID,Name,Artist,Album Artist,Album,Work,Genre,Total Time,Year,Date Added,Play Count,Play Date UTC,Sort Album,Sort Album Artist,Sort Artist,Persistent ID,Skip Count,Skip Date,Sort Name,Composer,Release Date,Compilation
0,3000,Everything's Ok [Pretty Mary K](David McConnel...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,278700,2002,2020-06-26T16:04:32Z,3,2020-08-08T17:39:03Z,Unreleased,Elliott Smith,Elliott Smith,4E0E450EECD74EDA,NaN,NaN,NaN,NaN,NaN,NaN
1,3003,A Distorted Reality Is Now a Necessity to Be F...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,190824,2002,2020-06-26T16:04:32Z,1,2020-06-27T02:44:48Z,Unreleased,Elliott Smith,Elliott Smith,85EEC4BE78C8DEC4,1,2020-08-15T17:56:36Z,Distorted Reality Is Now a Necessity to Be Fre...,NaN,NaN,NaN
2,3006,Suicide Machine (Fritz Machaud New Monkey Stud...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,182543,2003,2020-06-26T16:04:32Z,3,2020-06-27T02:41:37Z,Unreleased,Elliott Smith,Elliott Smith,748C7375168B1E23,1,2020-06-26T20:41:39Z,NaN,NaN,NaN,NaN
3,3009,Coast to Coast [Instrumental](David McConnell ...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,302132,2002,2020-06-26T16:04:32Z,1,2020-08-08T17:07:24Z,Unreleased,Elliott Smith,Elliott Smith,D1CC527ECAAE3EA0,NaN,NaN,NaN,NaN,NaN,NaN
4,3012,Don't Go Down [Acoustic Version](Jon Brion Two...,Elliott Smith,Elliott Smith,Unreleased,The Moon is a Sickle Cell,Music,215232,2001,2020-06-26T16:04:32Z,1,2020-08-08T16:54:16Z,Unreleased,Elliott Smith,Elliott Smith,A3CFFE0C010A0BA1,NaN,NaN,Don't Go Down,NaN,NaN,NaN


In [63]:
played_in_2020 = df[df['Play Date UTC']> '2020'].sort_values(by=['Play Date UTC'])
played_in_2020['Total Play Time'] = played_in_2020['Play Count'].astype('int32') * played_in_2020['Total Time'].astype('int32') / 60000

In [66]:
played_in_2020.sort_values(by=['Total Play Time'], ascending=False)

,Track ID,Name,Artist,Album Artist,Album,Work,Genre,Total Time,Year,Date Added,Play Count,Play Date UTC,Sort Album,Sort Album Artist,Sort Artist,Persistent ID,Skip Count,Skip Date,Sort Name,Composer,Release Date,Compilation,Total Play Time
13426,29918,Mothership Connection (Star Child),Parliament,Parliament,Mothership Connection,NaN,Funk,373267,1975,2020-02-26T19:04:45Z,18,2020-11-29T18:58:30Z,Mothership Connection,NaN,Parliament,044A087DE8630DCB,1,2020-06-13T18:04:18Z,Mothership Connection (Star Child),"George Clinton, Jr., William Earl Collins & Be...",1975-12-15T12:00:00Z,NaN,111.980100
15226,33535,Get Down,Curtis Mayfield,Curtis Mayfield,Roots,NaN,R&B/Soul,348333,1971,2020-08-19T21:24:54Z,12,2020-11-29T19:17:21Z,Roots,NaN,Curtis Mayfield,BAE668CBAB063CBF,NaN,NaN,Get Down,Curtis Mayfield,1971-10-01T07:00:00Z,NaN,69.666600
15225,33533,Beautiful Brother of Mine,Curtis Mayfield,Curtis Mayfield,Roots,NaN,R&B/Soul,447200,1971,2020-08-19T21:24:54Z,9,2020-11-29T19:11:32Z,Roots,NaN,Curtis Mayfield,85C39B286D15E618,NaN,NaN,Beautiful Brother of Mine,Curtis Mayfield,1971-10-01T07:00:00Z,NaN,67.080000
1784,6618,who sees you,My Bloody Valentine,NaN,M B V,NaN,Shoegaze,372271,2013,2016-04-20T06:11:29Z,9,2020-12-29T01:47:14Z,NaN,NaN,NaN,7EF3735E606D682B,NaN,2018-12-08T05:40:25Z,NaN,NaN,NaN,NaN,55.840650
13537,30140,Genius of Love,Tom Tom Club,Tom Tom Club,Tom Tom Club,NaN,Pop,334353,1981,2020-03-11T03:48:17Z,10,2020-11-29T19:04:05Z,Tom Tom Club,NaN,Tom Tom Club,0F40FE20931C38A5,NaN,NaN,Genius of Love,"Tom Tom Club, Chris Frantz & Tina Weymouth",1981-10-01T12:00:00Z,NaN,55.725500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13081,29228,creation day the travel flute way,Broadcast,NaN,Mother is the MIlky Way,NaN,Space,33959,2009,2018-03-26T03:45:21Z,1,2020-07-29T01:12:36Z,NaN,NaN,NaN,8610882D67213708,NaN,NaN,NaN,NaN,NaN,NaN,0.565983
10874,24798,Ostrich & Chirping,Elliott Smith,Elliott Smith,From A Basement On The Hill,NaN,Alternative,33926,2004,2017-12-23T19:49:58Z,1,2020-06-30T22:06:09Z,From A Basement On The Hill,NaN,Elliott Smith,967B7F6A583A29B6,NaN,2019-01-01T19:18:43Z,Ostrich & Chirping,NaN,2004-10-18T12:00:00Z,NaN,0.565433
3698,10446,You're Not An Airplane,Guided By Voices,NaN,Bee Thousand,NaN,Lo-Fi,33358,1994,2009-12-11T09:37:32Z,1,2020-09-19T23:07:46Z,NaN,NaN,NaN,C43AEADF0B493487,NaN,2017-12-25T07:36:29Z,NaN,Tobin Sprout,NaN,NaN,0.555967
13080,29226,growing backwards,Broadcast,NaN,Mother is the MIlky Way,NaN,Space,19670,2009,2018-03-26T03:45:21Z,1,2020-06-21T00:07:32Z,NaN,NaN,NaN,C50BC7A40955E48E,NaN,2018-03-30T21:47:30Z,NaN,NaN,NaN,NaN,0.327833
